In [3]:
# autoencoder/train_autoencoder.py

import numpy as np
import os
import wfdb
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# --- Load ECG signal ---
record = wfdb.rdrecord(os.path.join("../data", "100"))
signal = record.p_signal[:, 0]  # use first channel (MLII)

# --- Extract heartbeats using fixed-length windows ---
window_size = 200  # 200 samples ~0.55 seconds at 360Hz
stride = 50
segments = []

for i in range(0, len(signal) - window_size, stride):
    segment = signal[i:i + window_size]
    segments.append(segment)

segments = np.array(segments)
segments = segments.reshape((-1, window_size, 1))  # shape: (num_segments, 200, 1)

# --- Normalize ---
segments = (segments - np.min(segments)) / (np.max(segments) - np.min(segments))

# --- Split ---
X_train, X_test = train_test_split(segments, test_size=0.2, random_state=42)

# --- Autoencoder architecture ---
input_sig = Input(shape=(window_size, 1))

x = Conv1D(16, 3, activation="relu", padding="same")(input_sig)
x = MaxPooling1D(2, padding="same")(x)
x = Conv1D(8, 3, activation="relu", padding="same")(x)
x = MaxPooling1D(2, padding="same")(x)
x = Conv1D(8, 3, activation="relu", padding="same")(x)
encoded = MaxPooling1D(2, padding="same", name="bottleneck")(x)

x = Conv1D(8, 3, activation="relu", padding="same")(encoded)
x = UpSampling1D(2)(x)
x = Conv1D(8, 3, activation="relu", padding="same")(x)
x = UpSampling1D(2)(x)
x = Conv1D(16, 3, activation="relu")(x)
x = UpSampling1D(2)(x)
decoded = Conv1D(1, 3, activation="sigmoid", padding="same")(x)

autoencoder = Model(input_sig, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

# --- Train ---
autoencoder.fit(X_train, X_train, epochs=20, batch_size=32,
                validation_data=(X_test, X_test))  

# --- Save model ---
autoencoder.save("../models/ecg_autoencoder.h5")

print("Autoencoder training complete and saved.")


c:\Users\jains\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\jains\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\jains\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

Epoch 1/20


ValueError: Dimensions must be equal, but are 200 and 196 for '{{node compile_loss/mse/sub}} = Sub[T=DT_FLOAT](data_1, functional_1/conv1d_6_1/Sigmoid)' with input shapes: [?,200,1], [?,196,1].